In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab10.ipynb")

# Lab 10: Multiple Linear Regression and Feature Selection/Engineering

Welcome to Lab 11! In this lab we will introduce feature engineering, build multiple linear regression models, and investigate methods to evaluate the "goodness of fit" for linear models.

To receive credit for a lab, answer all questions correctly and submit before the deadline.

**Due Date:**

**Collaboration Policy:** Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others **please include their names below** (it's a good way to learn your classmates' names).

**Collaborators:** 

List collaborators here.

**Note:** In this notebook a custom figure size has been configured. Click [here](https://matplotlib.org/users/customizing.html) to read the documentation about customizing aspects of matplotlib.

Run the cell below.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from sklearn.linear_model import LinearRegression

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['figure.dpi'] = 100

# 1. Interpreting the Estimated Parameters

In Foundations of Data Science we learned about the regression line in section 15.2 of the textbook, Computational and Inferential Thinking. To refresh your memory about the concept click [here](https://inferentialthinking.com/chapters/15/2/Regression_Line.html#).

In regression with a single independent variable, the coefficient of the independent variable tells you how much the dependent variable is expected to increase (if the coefficient is positive) or decrease (if the coefficient is negative) when that independent variable increases by one. Interpreting the intercept in a regression model isn’t always straightforward. The intercept (often labeled the constant) is the expected mean value of the independent variable when the independent variable is 0. If the independent variable is never equal to 0, then the intercept has no meaning.

We will continue our analysis of the `mpg` data set.

**Note:** We load the full data set, and drop any rows that have missing data.

In [ ]:
mpg = sns.load_dataset('mpg').dropna()
mpg = mpg.sort_values('horsepower', ascending = True)
mpg.head()

In [ ]:
mpg_model = LinearRegression()
mpg_model.fit(mpg[['horsepower']], mpg['mpg'])
slope = mpg_model.coef_[0]
y_int = mpg_model.intercept_ 
print("The estimated parameter for the slope is", slope)
print("The estimated parameter for the y-intercept is", y_int)

What is the interpretation of the value of the slope, within the context of the modeling scenario?

_Type your answer here, replacing this text._

Does the intercept have meaning for this model? Explain.

_Type your answer here, replacing this text._

# 2. Re-expression

In the previous lab we showed how to establish relationships between one explanatory variable and a response variable. 

Let's review. Run the cell below to see a scatter plot of the data.

In [ ]:
sns.lmplot(x = 'horsepower', y = 'mpg', data = mpg, ci = None, line_kws = {'color': 'red'})
plt.xlabel("Horsepower")
plt.ylabel("mpg");

To straighten the data we re-expressed `mpg` by taking the square root.

In [ ]:
mpg['sqrt_mpg'] = np.sqrt(mpg['mpg'])
mpg.head()

Then we made a simple linear regression model using our re-expressed response `sqrt_mpg`.

In [ ]:
sqrt_mpg_model = LinearRegression()
sqrt_mpg_model.fit(mpg[['horsepower']], mpg['sqrt_mpg'])

We looked at the new scatter plot.

In [ ]:
sns.lmplot(x = 'horsepower', y = 'sqrt_mpg', data = mpg, ci = None, line_kws = {'color': 'red'})
plt.xlabel("Horsepower")
plt.ylabel("$\sqrt{mpg}$");

The dip in our scatter plot is not as noticeable.

Then we looked at our residual plot.

In [ ]:
sns.residplot(x = "horsepower", y = "sqrt_mpg", data = mpg)
plt.axhline(y = 0, color = 'r')
plt.xlabel("Horsepower")
plt.ylabel("Residuals");

Re-expressing the response seemed to improve our residual plot. However, we still haven't quite captured the curvature in data, especially at the ends. Let's add more features!

# 3. Multiple Linear Regression

With real-world problems you will often want to use **multiple features** to model and predict a response variable. To do so,you can use multiple linear regression. 

Multiple linear regression attempts to model the relationship between two or more explanatory variables and a response variable by fitting a linear equation to the observed data. Formally, the model for multiple linear regression, given $n$ features is:

$$y = \theta_0 + \theta_1 x_2 + \theta_2 x_2 + … + \theta_n x_n $$

where each $\theta_i$ is an estimated linear parameter. This is what is meant by the phrase *"linear in the parameters"*. If you change the equation to

$$y = \theta_0 + \theta_1 x_2^2 + \theta_2 x_2 + … + \theta_n x_n$$

Then, it is no longer linear in variables (because of the squared term $\theta_2 x_2^2$) but it is still linear in parameters. And for (multiple) linear regression, that's all that matters because in the end, you are trying to find a set of $\theta$s that minimizes a loss function [(Data Science Exchange)](https://datascience.stackexchange.com/questions/12274/what-does-linear-in-parameters-mean).

Let's look at an example.

# 4. Advertising

Suppose that we are statistical consultants hired by a client to provide advice on how to improve sales of a particular product. The `ads` data set consists of the sales (in thousands of units) of that product in 200 different markets, along with advertising budgets for the product in each of those markets for three different media: internet, streaming radio, and tv (all in thousands of dollars). 

It is not possible for our client to directly increase sales of the product. On the other hand, they can control the advertising expenditure in each of the three media. Therefore, if we determine that there is an **association between advertising and sales**, then we can instruct our client to adjust advertising budgets, thereby indirectly increasing sales. In other words, our goal is to develop an accurate model that can be used to predict sales on the basis of the three media budgets. [(An Introduction to Statistical Learning by Gareth James)](https://hastie.su.domains/ISLR2/ISLRv2_website.pdf) 

In [ ]:
ads = pd.read_csv("data/ads.csv",index_col = 0)
ads.head()

Use the three code cells below to perform EDA on the `ads` data set.

### EDA Part 1 of 3

In [ ]:
bins = np.arange(0, 35, 5)
plt.hist(ads["sales"], bins = bins, edgecolor = 'black')
plt.title("Distribtion of Sales");

### EDA Part 2 of 3

In [ ]:
ads.corr()

### EDA Part 3 of 3

In [ ]:
plt.scatter(x = ads.internet, y = ads.sales)
plt.xlabel("Internet")
plt.ylabel("Sales");

Now choose one feature, plot the residuals, fit a simple linear regression model, calculate the root mean squared error.

### SLR Step 1 of 4

Choose a single feature and plot the residuals.

In [ ]:
sns.residplot(x = "internet", y = "sales", data = ads)
plt.axhline(y = 0, color = 'r')
plt.xlabel("Internet")
plt.ylabel("Residuals");

### SLR Step 2 of 4

Fit a simple linear regression model using `sci-kit learn`.

In [ ]:
sales = LinearRegression()
sales.fit(ads[['internet']], ads['sales'])

### SLR Step 3 of 4

Calculate the root mean squared error.

In [ ]:
(np.sum(np.square(ads['sales'] - sales.predict(ads[['internet']]))/len(ads['sales'])))**.5

### SLR Step 4 of 4 

Write 2-3 sentences to asses your model. For example you could the meaning of your estimated parameters for the slope and $y-$intercept, you could interpret the meaning of your value for RMSE, etc.

**Note:** Use the blank code cell below for additional programming analysis. Type your sentences in the markdown cell labeled *Type your answer here, replacing this text.*.

_Type your answer here, replacing this text._

By limiting our model to one feature, we may be leaving out information that could produce a better model. In other words, the region sales could be affected by more than just the advertising money spent on internet ads. Let's fir another model with an additional feature, then compare it to our single feature model.

In [ ]:
sales2 = LinearRegression()
sales2.fit(ads[['internet', 'streaming_radio']], ads['sales'])
sales2_pred = sales2.predict(ads[['internet', 'streaming_radio']])

Now let's calculate the RMSE.

In [ ]:
(np.sum(np.square(ads['sales'] - sales2_pred))/len(ads['sales']))**(0.5)

Write 2-3 sentences to compare the `sales` model (simple linear regression using one predictor) to the `sales2` model (multiple linear regression using two predictors).

**Note:** Use the blank code cell below for additional programming analysis. Type your sentences in the markdown cell labeled *Type your answer here, replacing this text.*.

_Type your answer here, replacing this text._

**Question 1.** Given the equation

$$\text{sales}=\hat{\theta}_0+\hat{\theta}_1 \times \text{internet}+\hat{\theta}_2 \times \text{streaming_radio},$$

find the values for $\hat{\theta}_0$, $\hat{\theta}_1$, and $\hat{\theta}_2$ (rounded to 4 decimal places).

**Hint:** In Lab 10 we used the `.coef_` and the `.intercept_` methods on our model instance to retrieve the estimated parameters of our simple linear regression model.

In [ ]:
theta0 = ...
theta1 = ...
theta2 = ...

In [ ]:
grader.check("q1")

<!-- BEGIN QUESTION -->

**Question 2.** Using the values for $\hat{\theta}_0$, $\hat{\theta}_1$, and $\hat{\theta}_2$ (rounded to 4 decimal places), in Latex ($\LaTeX$), write out the function that the model `sales2` is using to predict `sale` from `internet` and `streaming_radio`.

**Hint:** Click [here](http://tug.ctan.org/info/undergradmath/undergradmath.pdf) to view a brief two page guide on typesetting math equations in Latex. If you still need help, as a neighbor or your instructor.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Interpreting the Model Parameters

- Each  parameter represents the change in the mean response, $\hat y$, per unit increase in the associated predictor variable when all the other predictors are held constant.

- For example,  $\theta_1 = 0.0458$, represents the estimated change in the mean response, $\hat y$, per unit increase in the amount spent on internet advertising when the amount spent on streaming radio advertisement is held constant.

- The intercept term, $\theta_0 = 2.9211$, represents the estimated mean response, $\hat y$, when all the predictors (amount spent in advertising on internet and streaming radio) are all zero (which may or may not have any practical meaning).

<!-- BEGIN QUESTION -->

**Question 3.** In our situation would it make sense to interpret the intercept term? Write 2-3 sentences to explain your answer. 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

# 5. R Squared

$R^2$, denoted by

$$R^2 = \frac{\text{variance of fitted values}}{\text{variance of true } y} = \frac{\hat{\sigma}_{y}^2}{\sigma_y^2}$$

is a goodness-of-fit measure for linear regression models.

This statistic indicates the percentage of the variance in the dependent variable that the independent variable(s) explain collectively. R-squared measures the strength of the relationship between your model and the dependent variable on a convenient 0$-$100% scale.

Unlike $r$, the correlation coefficient we looked at in Lab 10, $R^2$, the **coefficient of determination**, can be used in the multiple regression setting. In simple regression, $r^{2}$ and Multiple $R^{2}$ are the same.

To see exactly how much better our new model is, we can compare the Multiple $R^2$ from the simple linear and multiple linear fits.

## Which Metric Should You Use?

When assessing how well a model fits a data set, it’s useful to calculate both the RMSE and the $R^2$ value because each metric tells us something different.

- One one hand, RMSE tells us the typical distance between the predicted value made by the regression model and the actual value.

- On the other hand, $R^2$ tells us how well the predictor variables can explain the variation in the response variable.

**Question 4.** Calculate the R-squared value for both the `sales` and `sales2` model.

In [ ]:
r2_sales = ...
r2_sales2 = ...
r2_sales, r2_sales2

In [ ]:
grader.check("q4")

## Limitations of R-squared

- R-squared can't determine whether the coefficient estimates ($theta_i$s) and predictions ($\hat y_i$s) are biased, which is why you must assess the residual plots.

- R-squared does not indicate whether a regression model is adequate. You can have a low R-squared value for a good model, or a high R-squared value for a model that does not fit the data.

With this in mind let's examine our residual plots.

In [ ]:
plt.figure(figsize = [10, 15])

plt.subplot(3, 1, 1)
sns.residplot(x = "internet", y = "sales", data = ads)
plt.axhline(y = 0, color = 'r')
plt.xlabel('Internet')
plt.ylabel("Residuals")

plt.subplot(3, 1, 2)
sns.residplot(x = "streaming_radio", y = "sales", data = ads)
plt.axhline(y = 0, color = 'r')
plt.xlabel('Streaming Radio')
plt.ylabel("Residuals")

plt.subplot(3, 1, 3)
plt.scatter(sales2_pred, ads['sales'] - sales2_pred)
plt.axhline(y = 0, color = 'r')
plt.xlabel('Predicted Sales')
plt.ylabel('Residuals');

Our residual plot for `Predicted Sales` has a dip. This indicates a nonlinear relationship in the original data set. We could try re-expression, fitting a nonlinear model, etc. Since the goal of this unit in the course is to introduce you to the idea of statistical modeling, techniques beyond what we have discussed will not be explored. 

Every time you add a predictor to a model, the R-squared increases, even if due to chance alone. It never decreases. Consequently, a model with more terms may appear to have a better fit simply because it has more terms. If a model has too many predictors, it begins to model the random noise in the data. This condition is known as overfitting the model and it produces misleadingly high R-squared values and a lessened ability to make predictions.

R-squared is a handy, seemingly intuitive measure of how well your linear model fits a set of observations. However, R-squared doesn’t tell us the entire story. You should evaluate R-squared values in conjunction with residual plots, other model statistics (not part of this course), and subject area knowledge when choosing a statistical model.

# 6. Open Analysis

Now it's your turn to develop a multiple linear regression model. Use the `mpg` data set and fit a multiple linear regression model (mlr) on more than one of the numerical predictors. Keep in mind how residual plots, RMSE, and R-squared can be used to help you compare the models you fit. Fit at least two mlr models using different predictors. Compare your results, choose one of your models, and interpret your results from the model you choose. Make sure you provide written commentary as your perform your analysis.

**Note:** This part of the notebook will count 15 points towards the total grade of 20. It will be graded using the following rubric:

|**Description**|**Points**|
|---------------|----------|
|EDA (numerical and graphical analysis)| 5|
|Model development|5|
|Written Commentary|5|

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

When done exporting, download the .zip file by `SHIFT`-clicking on the file name and selecting **Save Link As**. Or, find the .zip file in the left side of the screen and right-click and select **Download**. You'll submit this .zip file for the assignment in Canvas to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)